In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
import time
import textwrap

from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain.schema.runnable import RunnablePassthrough

loader = WebBaseLoader("https://teddylee777.github.io/data-science/optuna/")
docs = loader.load()
content = docs[0].page_content

prompt = hub.pull("lawwu/chain_of_density")
json_parser = SimpleJsonOutputParser()

In [7]:
chain = (
    {"ARTICLE": RunnablePassthrough()}
    |prompt
    |ChatOpenAI(model="gpt-3.5-turbo-16k",temperature=0.1)
    |json_parser
)
chain.invoke(content)

[{'Missing_Entities': 'Optuna',
  'Denser_Summary': 'This article discusses Optuna, a hyperparameter optimization framework. Optuna is a powerful tool for optimizing machine learning models. It provides various methods for suggesting hyperparameters, such as suggest_categorical, suggest_int, and suggest_float. By using Optuna, you can efficiently search for the best set of hyperparameters for your model.'},
 {'Missing_Entities': 'study',
  'Denser_Summary': 'This article discusses Optuna, a hyperparameter optimization framework. Optuna provides a study object that allows you to optimize your model by suggesting hyperparameters. You can create a study object and use the optimize() method to search for the best set of hyperparameters. The study object keeps track of the trials and their results, allowing you to easily analyze and compare different hyperparameter configurations.'},
 {'Missing_Entities': 'objective function',
  'Denser_Summary': 'This article discusses Optuna, a hyperparam

In [13]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.base import BaseCallbackHandler
import json

loader = WebBaseLoader(
    "https://n.news.naver.com/mnews/article/016/0002339719"
)

docs = loader.load()
content = docs[0].page_content

class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs):
        print(token,end="",flush=True)

prompt = ChatPromptTemplate.from_template(
    """Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article. 

Repeat the following 2 steps 5 times. 

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

A missing entity is:
- relevant to the main story, 
- specific yet concise (50 words or fewer), 
- novel (not in the previous summary), 
- faithful (present in the article), 
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. 
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities. 

Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
Use only KOREAN language to reply."""
)


chain = (
    prompt
    | ChatOpenAI(
        temperature=0,
        model="gpt-4-turbo-preview",
        streaming=True,
        callbacks=[StreamCallback()],
    )
    | JsonOutputParser()
    | (lambda x: x[-1]["Denser_Summary"])
)

result = chain.invoke({"ARTICLE": content})
print(result)

```json
[
    {
        "Missing_Entities": "",
        "Denser_Summary": "이 기사는 최근 배달의민족 앱 사용자 수가 급감한 현상에 대해 다루고 있습니다. 이러한 사용자 이탈은 주로 수수료 인상으로 인한 음식 가격 상승 때문으로 보입니다. 특히 20대 사용자의 이탈이 두드러지며, 이는 배달의민족 앱의 주간 활성 이용자 수가 올해 최저치를 기록한 것과 관련이 있습니다. 배달의민족은 수수료를 인상하겠다고 발표한 후, 이에 대한 역풍을 맞고 있으며, 이는 최종적으로 소비자에게 피해가 돌아가는 상황입니다. 이와 관련하여 외식업주들과 시민단체들은 수수료 인상에 대해 규탄하고 있으며, 배달의민족 내부에서도 수수료 인상 결정에 대한 비판의 목소리가 나오고 있습니다."
    },
    {
        "Missing_Entities": "배달의민족 중개 수수료 인상안; 독일 딜리버리히어로(DH) 모기업; 공정거래위원회 현장 조사",
        "Denser_Summary": "배달의민족 앱 사용자 수가 급감하고 있으며, 이는 수수료 인상으로 인한 음식 가격 상승 때문입니다. 특히 20대 사용자의 이탈이 두드러지며, 배달의민족은 중개 수수료 인상안 발표 후 역풍을 맞고 있습니다. 이로 인해 외식업주들과 시민단체들이 수수료 인상을 규탄하고 있으며, 독일 딜리버리히어로(DH) 모기업에 대한 비판도 제기되고 있습니다. 또한, 공정거래위원회는 배달의민족에 대한 현장 조사를 진행하고 있습니다."
    },
    {
        "Missing_Entities": "배달의민족 앱 주간 활성 이용자 수 최저치 기록; 20대 이용자수 약 7% 감소",
        "Denser_Summary": "배달의민족 앱 사용자 수가 급감, 특히 20대 이탈이 두드러지며 주간 활성 이용자 수가 최저치를 기록했습니다. 이는 수수료 인상으로 인한 음식 가격 상승 때문이며, 중개 수수료 인상안 발표 후 역풍을 맞고 있습니다. 외식업